In [68]:
import pandas as pd

In [69]:
import numpy as np

In [70]:
df = pd.read_excel("sp500.xlsx")

In [71]:
df['Date'] = df['Date'].astype('datetime64')

In [72]:
df.set_index("Date", inplace=True)

In [73]:
df.sort_index(inplace=True)

In [74]:
df.fillna(method="ffill", inplace=True)

In [75]:
dailyret = df.pct_change()

## Calculate daily returns of long worst 5 strategy
1. buy the assets with the worst 5 returns at close (at open the next day)
2. Sell at close the next day
Assumes no slippage between close[t] and open[t+1]

In [76]:
trades = np.zeros(df.shape[0])

In [77]:
for t in np.arange(1, df.shape[0] - 2):
    worst5 = dailyret.iloc[t].nsmallest(5).index
    trades[t + 1] = df.iloc[t + 1][worst5].sum() - df.iloc[t][worst5].sum()

In [78]:
# average daily return
trades[2:].mean()

-0.15417849999999494

In [79]:
# annualized Sharpe ratio, assuming 4% annual return on risk-free basis
trades_dailyret = pd.Series(trades[1:]).pct_change()

In [80]:
trades_dailyret_net = trades_dailyret - 0.04 / 252

In [81]:
sharpe = np.sqrt(252) * np.mean(trades_dailyret_net) / np.std(trades_dailyret_net)

In [82]:
sharpe

nan

In [83]:
trades_dailyret_net

0            NaN
1           -inf
2     -35.779056
3      -0.759083
4      -0.732918
         ...    
244    -1.522895
245     0.029243
246    -0.839444
247    -3.376224
248    -1.000159
Length: 249, dtype: float64

In [84]:
trades_dailyret_net.mean()

-inf

In [85]:
trades_dailyret_net.mean() / trades_dailyret_net.std()

nan

In [86]:
np.sqrt(250) * (trades_dailyret_net.mean() / trades_dailyret_net.std())

nan

## read SPX index returns over 250 days
### Get excess returns from "long worst 5" strategy by evaluating it directly against actual returns from the S&P 500 index

In [87]:
spx = pd.read_csv("^SPX.csv")

In [88]:
spx['Date'] = spx['Date'].astype('datetime64')

In [89]:
spx.set_index('Date', inplace=True)

In [90]:
spx.sort_index(inplace=True)

In [91]:
riskfree_ret = spx['Adj Close'].pct_change().mean()

In [92]:
excessret = trades_dailyret - riskfree_ret

In [93]:
sharpe_ratio = np.sqrt(252) * excessret.iloc[2:].mean() / excessret.iloc[2:].std()

In [94]:
sharpe_ratio

-1.7209194545655262